In [1]:
# preprocess data
import numpy as np
import pandas as pd


import re
import copy 
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

from IOHMM import UnSupervisedIOHMM
from IOHMM import OLS, DiscreteMNL, CrossEntropyMNL
import random


In [2]:
# construct the dataset
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from sklearn.model_selection import train_test_split
def assign_task(n):
    sequence = np.random.random(n)
    sequence[sequence>0.5] = 1
    sequence[sequence<=0.5] = 0
    return sequence
    
def covxy(x,y):
    if type(x) == list:
        x = np.array(x)
    if type(y) == list:
        y = np.array(y)
    covxy = np.mean((x - x.mean()) * (y - y.mean()))
    return covxy


N =1000
# analyze human reliance prob
query = pd.read_csv("query_.csv")
data = pd.read_csv("predictions_0110.csv")
data["taskId"] = data["taskId"].apply(lambda x:int(x))
data = data[data["taskId"]<16]
data["human_confidence"] = data["imageName"].apply(lambda x:query.query("Name==@x")["Confidence_Pred_val"].item())
data["human_confidence"] = data["human_confidence"].apply(lambda x:1 if x>0.5 else 0)
selected_data = data[data["attackType"]==0]
workers = list(set(selected_data["workerId"]))
h_a = []
h_n = []
l_a = []
l_n = []
for w in workers:
    d = data.query("workerId==@w")
    h_num = len(d.query("human_confidence==1")["reliance"])
    l_num = len(d.query("human_confidence==0")["reliance"])
    h_a.append(d.query("human_confidence==1 and attack==0")["reliance"].sum()/h_num)
    h_n.append(d.query("human_confidence==1 and attack==-1")["reliance"].sum()/h_num)
    l_a.append(d.query("human_confidence==0 and attack==0")["reliance"].sum()/l_num)
    l_n.append(d.query("human_confidence==0 and attack==-1")["reliance"].sum()/l_num)
mean = torch.tensor([np.mean(l_a),np.mean(h_a),np.mean(l_n),np.mean(h_n)])
var = torch.tensor([[covxy(l_a,l_a),covxy(l_a,h_a),covxy(l_a,l_n),covxy(l_a,h_n)],[covxy(h_a,l_a),covxy(h_a,h_a),covxy(h_a,l_n),covxy(h_a,h_n)],
                   [covxy(l_n,l_a),covxy(l_n,h_a),covxy(l_n,l_n),covxy(l_n,h_n)],[covxy(h_n,l_a),covxy(h_n,h_a),covxy(h_n,l_n),covxy(h_n,h_n)]])

dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())

task_pool = assign_task(n=20)

In [3]:
dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())
train_human, test_human, _, _ = train_test_split(humans,[0 for i in range(len(humans))], test_size=0.1, random_state=2023)
test_humans = [np.concatenate((i,i),0) for i in test_human]
# baselines [low_attack, high_attack, low_no, high_No]
W_a = [-1,-0.5,0]
W_r = [0.25,0.75,1]
C = np.linspace(0,1,11)
# no attack
def no_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - attack * c


def all_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = n
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - n*len(humans) * c

def half_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
   
    for h in humans:
        tasks = assign_task(n)
        p = np.random.choice([i for i in range(n)],n//2, replace =None)
        attack_p = np.zeros(n)
        attack_p[p] = 1
        for k, t in enumerate(tasks):
            if t == 0 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and attack_p[k] == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]])  
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack*c

def high_confidence_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
                attack += 1
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack * c


def eval_func(func, n, w_a, w_r, c, humans, num=50):
    u = []
    for _ in range(num):
        u.append(func(n, w_a,w_r,c,humans))
    return sum(u)/num
    
    


In [8]:
from statsmodels.tools import add_constant

def trust_transition(model, inputs, state_src, state_dst):
#     inputs = add_constant(inputs,has_constant='add')
    prob = np.exp(model.model_transition[state_src].predict_log_proba(inputs)).reshape(model.num_states)
    return prob[state_dst]

def reliance_prob(model, inputs, state):
#     inputs = add_constant(inputs,has_constant='add')

    return np.exp(model.model_emissions[state][0].predict_log_proba(inputs)).reshape(2)


def reward(model, state, confidence,attack, w_a, w_r, c):
    inputs = np.array([[confidence, attack]])

    r = 0
    for i in range(model.num_states):

        r += trust_transition(model, inputs, state, i)* (reliance_prob(model, inputs,i)[0]*w_r +
                                                         reliance_prob(model, inputs,i)[1]*w_a - attack*c)
    return r
        
def reward_belief(model, belief, confidence,attack, w_a, w_r, c):
    r = 0
    for i in range(belief.shape[0]):
        r += belief[i] * reward(model, i, confidence, attack, w_a, w_r, c)
    return r

def EU_max(model, belief, confidence, attack, l, w_a, w_r, c,max_length=2):
    if l < 1:
        print("error: l should be greated than 1")
    if l == 1:
        eu_max = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        return eu_max
    else:
        r = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        eu_max = r 
        for d in [0,1]:
            sum_ = 0
            updated_b = update_belief(model, belief, confidence, attack,reliance=d)
            for k in range(belief.shape[0]):
                b_k = belief[k]
                inputs = np.array([[ confidence, attack]])
                for i in range(model.num_states):
                    
                    sum_ += b_k*trust_transition(model, inputs, k, i)*(reliance_prob(model,inputs,i)[d]*max(0.5*(EU_max(model, updated_b,confidence=0,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)
                                                                                                              +EU_max(model, updated_b,confidence=1,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)),
                                                                                    0.5*(EU_max(model, updated_b,confidence=0,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)+EU_max(model, updated_b,confidence=1,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c))))
            eu_max += sum_
        return eu_max

def update_belief(model,belief, confidence, attack, reliance, constant = 1):
    updated_b = np.zeros_like(belief)
    inputs = np.array([[confidence, attack]])
    for j in range(belief.shape[0]):
        for i in range(belief.shape[0]):
            updated_b[j] += constant*belief[i]*trust_transition(model, inputs, state_src=i,state_dst=j)*reliance_prob(model, inputs, j)[reliance]
    return updated_b
    
def covxy(x,y):
    if type(x) == list:
        x = np.array(x)
    if type(y) == list:
        y = np.array(y)
    covxy = np.mean((x - x.mean()) * (y - y.mean()))
    return covxy

def hmm_attack_helper(model, n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        b = np.array([1/model.num_states for i in range(model.num_states)])

        tasks = assign_task(n)
        for k, t in enumerate(tasks):
            
            action = np.argmax([EU_max(model, b, t, 0, n-k, w_a, w_r, c),EU_max(model, b, t, 1, n-k, w_a, w_r, c)])
            
            if t == 0 and action==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and action==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and action == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]]) 
            b = update_belief(model, b, t,action, d)
            if d == 1:
                accept += 1
            else:
                reject += 1
            print(accept, reject, attack)
    return w_a * accept + w_r * reject - attack*c
    
    
    
    

In [15]:
#eval hmm-based algorithm

#1. contruct human behavior data
hmm_data = selected_data
behavior_data = []
hmm_data["reliance"] = hmm_data["reliance"].apply(lambda x: 1 if x == True else -1)
hmm_data["attack"] = hmm_data["attack"].apply(lambda x:1 if x == 0 else x)
hmm_data["human_confidence"] = hmm_data["human_confidence"].apply(lambda x:-1 if x == 0 else x)
workers = list(set(selected_data["workerId"]))
for w in workers:
    d = hmm_data.query("workerId==@w").sort_values(by=["taskId"])
    behavior_data.append(d[["human_confidence","attack", "reliance"]])
initial_columns = ["human_confidence", "attack"]
transition_columns = ["human_confidence", "attack"]
decision_columns = ["human_confidence", "attack"]

model = UnSupervisedIOHMM(num_states=2, max_EM_iter=200, EM_tol=1e-6)

model.set_models(model_emissions = [DiscreteMNL(solver='lbfgs')], 
                model_transition=CrossEntropyMNL(solver='lbfgs'),
                model_initial=CrossEntropyMNL(solver='lbfgs'))

model.set_inputs(covariates_initial = initial_columns, covariates_transition = initial_columns, covariates_emissions = [initial_columns])

model.set_outputs([['reliance']])

model.set_data(behavior_data)
model.train()
# hmm_attack_helper(model, 20,0,0.5,0.15, test_humans)

In [18]:
C = [i *0.01 for i in range(10)]

for c in C:
    print("attack cost:---->{}".format(c))
    print("high confidence:{}".format(eval_func(high_confidence_attack_helper,20,0,0.5,c, test_humans)))
    print("half attack:{}".format(eval_func(half_attack_helper,20,0,0.5,c, test_humans)))
    print("all attack:{}".format(eval_func(all_attack_helper,20,0,0.5,c, test_humans)))
    print("no attack:{}".format(eval_func(no_attack_helper, 20,0,0.5,c, test_humans)))

attack cost:---->0.0
high confidence:271.23
half attack:305.48
all attack:396.2
no attack:213.44
attack cost:---->0.01
high confidence:264.4069999999999
half attack:301.67000000000024
all attack:386.0900000000004
no attack:213.06
attack cost:---->0.02
high confidence:260.6772
half attack:295.91000000000025
all attack:377.94
no attack:214.85
attack cost:---->0.03
high confidence:255.65419999999995
half attack:290.75999999999976
all attack:367.4700000000001
no attack:215.73
attack cost:---->0.04
high confidence:248.74720000000005
half attack:286.9699999999998
all attack:356.8399999999997
no attack:215.68
attack cost:---->0.05
high confidence:248.19
half attack:279.54
all attack:345.65
no attack:215.05
attack cost:---->0.06
high confidence:241.00839999999997
half attack:274.11000000000024
all attack:336.6700000000003
no attack:216.5
attack cost:---->0.07
high confidence:237.26999999999995
half attack:272.7900000000002
all attack:327.22999999999973
no attack:214.88
attack cost:---->0.08
hi

In [21]:
C = [i *0.01 for i in range(1,10)]

In [ ]:
for c in C:
    print("cost:{}, utility:{}".format(c, hmm_attack_helper(model, 20,0,0.5,c, test_humans)))

0 1 1
0 2 2
0 3 3
0 4 4
0 5 5
1 5 6
1 6 7
1 7 8
1 8 9
1 9 10
1 10 11
1 11 12
1 12 13
1 13 14
1 14 15
2 14 16
3 14 17
3 15 18
4 15 19
4 16 20
4 17 21
4 18 22
4 19 23
4 20 24
4 21 25
4 22 26
5 22 27
5 23 28
6 23 29
6 24 30
7 24 31
7 25 32
7 26 33
7 27 34
7 28 35
7 29 36
7 30 37
7 31 38
8 31 39
8 32 40
8 33 41
8 34 42
8 35 43
8 36 44
9 36 45
9 37 46
10 37 47
11 37 48
11 38 49
11 39 50
11 40 51
11 41 52
12 41 53
12 42 54
13 42 55
13 43 56
13 44 57
13 45 58
13 46 59
13 47 60
13 48 61
13 49 62
13 50 63
13 51 64
13 52 65
14 52 66
14 53 67
14 54 68
15 54 69
15 55 70
16 55 71
16 56 72
16 57 73
16 58 74
16 59 75
16 60 76
16 61 77
17 61 78
17 62 79
18 62 80
18 63 81
18 64 82
19 64 83
19 65 84
19 66 85
19 67 86
19 68 87
20 68 88
20 69 89
21 69 90
22 69 91
23 69 92
23 70 93
24 70 94
24 71 95
25 71 96
26 71 97
26 72 98
26 73 99
26 74 100
26 75 101
26 76 102
26 77 103
26 78 104
27 78 105
27 79 106
28 79 107
29 79 108
29 80 109
29 81 110
29 82 111
29 83 112
30 83 113
30 84 114
30 85 115
30 86 116
30 8

155 590 745
155 591 746
155 592 747
155 593 748
155 594 749
156 594 750
156 595 751
156 596 752
156 597 753
156 598 754
156 599 755
156 600 756
156 601 757
156 602 758
156 603 759
156 604 760
156 605 761
156 606 762
156 607 763
156 608 764
156 609 765
156 610 766
156 611 767
156 612 768
156 613 769
156 614 770
156 615 771
156 616 772
156 617 773
156 618 774
156 619 775
156 620 776
156 621 777
156 622 778
156 623 779
157 623 780
157 624 781
157 625 782
157 626 783
157 627 784
157 628 785
157 629 786
157 630 787
157 631 788
157 632 789
157 633 790
157 634 791
157 635 792
157 636 793
157 637 794
157 638 795
157 639 796
157 640 797
157 641 798
157 642 799
157 643 800
158 643 801
158 644 802
158 645 803
158 646 804
159 646 805
160 646 806
160 647 807
160 648 808
160 649 809
160 650 810
161 650 811
162 650 812
162 651 813
162 652 814
162 653 815
163 653 816
163 654 817
164 654 818
164 655 819
165 655 820
165 656 821
165 657 822
165 658 823
165 659 824
165 660 825
165 661 826
165 662 827
165 